<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/lecture-idea/10_root_finding/42_examples_beams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Examples on deflections of the beams<br>보의 처짐에 응용한 사례



ref : Pytel et. al, Mechanics of Materials, 2nd Ed, SI, Ex. 6.9 and Tables 6.2 & 6.3, Cengage Learning, 2011.



In [ ]:
import pylab as py
import scipy.optimize as so
import sympy as sym
import sympy.utilities as su



In [ ]:
sym.init_printing()



## Constants for the material & the section<br>재료와 단면에 관한 상수



$$
\begin{align}
    EI&=20.48\times 10^3 N \cdot m^2 \\ 
    w_o&=100N/m \\ 
    L&=1m
\end{align}
$$



## 분포하중 외팔보 <br>Distributed load cantilever



### 질문<br>Question



The deflection of a uniform distributed load cantilver is as follows.<br>
균일 분포하중 외팔보의 처짐은 다음과 같다.



$$
\begin{align}
\delta(x) &= \frac{w_o x^2}{24EI}\left(6L^2 - 4Lx + x^2 \right) \\
\delta(L) &= \frac{w_o L^4}{8EI}
\end{align}
$$



Using following parameters, find $x$ such that its deflection $\delta(x)$ is the half of the deflection at $x=L$.<br>
다음 매개변수들을 사용하여 처짐이 $x=L$ 에서의 최대 처짐의 절반인 $x$(m)를 구하시오.



### 풀이 예<br>One possible answer



$$
\begin{align}
\delta(x) &= \frac{1}{2}\delta(L) \\
\frac{w_o x^2}{24EI}\left(6L^2 - 4Lx + x^2 \right) &= \frac{1}{2}\frac{w_o L^4}{8EI} \\
\frac{x^2}{3}\left(6L^2 - 4Lx + x^2 \right) &= \frac{1}{2}L^4
\end{align}
$$



$$
\frac{x^2}{3}\left(6L^2 - 4Lx + x^2 \right) - \frac{1}{2}L^4 = 0
$$



In [ ]:
L_m = 1



$g(x)=0 $



In [ ]:
def g(x_m):
    return (x_m ** 2) * (6 * L_m * L_m - 4 * L_m * x_m + x_m ** 2) / 3. - 0.5 * L_m ** 4



이분법<br>
Bisection Method



In [ ]:
%%time 
x_bis_m = so.bisect(g, a=0, b=L_m, xtol=1e-3)
print(x_bis_m)



뉴튼랩슨법<br>
Newotn Raphson Method



$$
\begin{align}
g(x) &= \frac{x^2}{3}\left(6L^2 - 4Lx + x^2 \right) - \frac{1}{2}L^4 \\
&= \left(6L^2\frac{x^2}{3} - 4Lx\frac{x^2}{3} + x^2\frac{x^2}{3} \right) - \frac{1}{2}L^4 \\
&= \frac{1}{3}x^4 - \frac{4L}{3}x^3 + 2L^2x^2 - \frac{1}{2}L^4 \\
\frac{d}{dx}g(x) &= \frac{4}{3}x^3 - 4Lx^2 + 4L^2x
\end{align}
$$



In [ ]:
def dg(x_m):
    return 4./3 * x_m ** 3 - 4 * L_m * x_m ** 2 + 4 * L_m ** 2 * x_m



In [ ]:
%%time
x_nr_m = so.newton(g, .1, fprime=dg, tol=1e-3)
print(x_nr_m)



## Simply supported beam under a concentrated load at its center<br>중점에 집중하중을 받는 단순지지보



### 질문<br>Question



Find $x$(m) such that its deflection $\delta(x)$ is the half of the maximum deflection at $x=\frac{L}{2}$.<br>
처짐 $\delta(x)$ 가 $x=\frac{L}{2}$ 에서의 최대 처짐의 절반인 $x$(m)를 구하시오.



### 풀이 예<br>One possible answer



Prepare symbols to use<br>사용할 기호를 준비



In [ ]:
P, x, EI, L = sym.symbols("P x EI L", real=True, nonnegative=True)



Deflection $\delta(x)$ within the interval $ 0 \le x \le \frac{L}{2}$<br>$ 0 \le x \le \frac{L}{2}$ 구간에서의 처짐 $\delta(x)$



In [ ]:
delta = (P * x)  * (3 * L * L  - 4 * x * x) / (48 * EI)
delta



Maximum deflection at the center $\delta\left( \frac{L}{2} \right)$<br>
가운데 지점에서의 최대 처짐$\delta\left( \frac{L}{2} \right)$



In [ ]:
delta_max = delta.subs({x: L/2})
delta_max



Equation to solve<br>풀고자 하는 방정식



In [ ]:
eq_p = sym.Eq(delta, delta_max / 2)
eq_p



Cancellation<br>약분



In [ ]:
factor = 48 * 2 * EI / P
eq_p = sym.Eq(eq_p.lhs * factor, eq_p.rhs * factor)
eq_p



Transposition<br>이항



In [ ]:
h_x = eq_p.lhs - eq_p.rhs
h_x



Substituting $L=1$, generating a python function<br>$L=1$을 대입하면서 파이썬 함수 생성



In [ ]:
h = sym.lambdify(x, h_x.subs({L:1}))



#### Bisection Method<br>이분법



In [ ]:
%%time 
x_bis_m = so.bisect(h, a=0, b=L_m*0.5, xtol=1e-3)
print(x_bis_m)



#### Newotn Raphson Method<br>뉴튼 랩슨법



Differentiate over $x$<br>$x$에 대해 미분



In [ ]:
dh_dx = h_x.diff(x)
dh_dx



Substituting $L=1$, generating a python function<br>$L=1$을 대입하면서 파이썬 함수 생성



In [ ]:
dh = sym.lambdify(x, dh_dx.subs({L:1}))



In [ ]:
%%time
x_nr_m = so.newton(h, .01, fprime=dh, tol=1e-3)
print(x_nr_m)



## Final Bell<br>마지막 종



In [ ]:
# stackoverfow.com/a/24634221
import os
os.system("printf '\a'");

